In [1]:
from numpy import sqrt, zeros, exp, linspace, meshgrid, array
from numpy.random import rand, choice

In [2]:
# Constants.
wh2_density = 1000.0 # per square micron
actin_conc = 5.0
diff_rate_const = 1.0 / (2.7e-3)**2
elongation_rate_const = 11.0
elongation_rate = elongation_rate_const * actin_conc
wh2_loading_rate_const = 5.0
loading_rate = wh2_loading_rate_const * actin_conc * (42 / (42 + 8))**(1/3) # Correct for dimension.
unloading_rate = 3.0 # per second

In [3]:
# Setup simulation.
no_wh2s_line = int(sqrt(wh2_density))
no_wh2s = no_wh2s_line**2
npf_state_mat = zeros((no_wh2s_line, no_wh2s_line), dtype = bool)
current_time = 0.0
time_interval = 1e-5
total_time = 10.0

no_domains_line = int(1.0 / 2.7e-3)
monomer_count_mat = zeros((no_domains_line, no_domains_line), dtype = int)
index_wh2_row = linspace(0, no_domains_line - 1, no_wh2s_line, dtype = int)
index_wh2_row_mat, index_wh2_col_mat = meshgrid(index_wh2_row, index_wh2_row, indexing = 'ij')
has_wh2_mat = zeros(monomer_count_mat.shape)
has_wh2_mat[index_wh2_row_mat, index_wh2_col_mat] = True

# Barbed end.
end_position = array([1, 0])

In [4]:
# Iterate.
while current_time <= total_time and end_position[1] < (no_domains_line - 2):
    # Check barbed end.
    if exp(-elongation_rate * time_interval) < rand():
        end_position[1] += 1
    if monomer_count_mat[end_position[0], end_position[1] + 1] > 0:
        monomer_count_mat[end_position[0], end_position[1] + 1] -= 1
        end_position[1] += 1
    # Diffuse free monomers.
    do_move_mat = exp(-diff_rate_const * monomer_count_mat * time_interval) < rand(no_domains_line, no_domains_line)
    monomer_count_mat[do_move_mat] -= 1
    index_move_row_mat, index_move_col_mat = do_move_mat.nonzero()
    no_moves = index_move_row_mat.size
    for i in range(no_moves):
        direction = choice(4)
        if direction == 0:
            if index_move_col_mat[i] < (no_domains_line - 1):
                monomer_count_mat[index_move_row_mat[i], index_move_col_mat[i] + 1] += 1
        elif direction == 1:
            if index_move_row_mat[i] > 0:
                monomer_count_mat[index_move_row_mat[i] - 1, index_move_col_mat[i]] += 1
        elif direction == 2:
            if index_move_col_mat[i] > 0:
                monomer_count_mat[index_move_row_mat[i], index_move_col_mat[i] - 1] += 1
        elif index_move_row_mat[i] < (no_domains_line - 1):
            monomer_count_mat[index_move_row_mat[i] + 1, index_move_col_mat[i]] += 1
    # Check NPFs for loading and unloading.
    rand_mat = rand(no_wh2s_line, no_wh2s_line)
    loading_prob_mat = rand_mat * (npf_state_mat == False)
    unloading_prob_mat = rand_mat * (npf_state_mat == True)
    npf_state_mat[exp(-loading_rate * time_interval) < loading_prob_mat] = True
    is_unloading_likely_mat = exp(-unloading_rate * time_interval) < unloading_prob_mat
    npf_state_mat[is_unloading_likely_mat] = False
    monomer_count_mat[index_wh2_row_mat[is_unloading_likely_mat], index_wh2_col_mat[is_unloading_likely_mat]] += 1
    current_time += time_interval

In [6]:
print(end_position / current_time)

[ 0.17855995 65.7100615 ]
